In [ ]:
import pandas as pd
import csv
import pyproj
import math
import numpy as np
from math import sin, cos, sqrt, atan2, radians

df_arduino = pd.read_csv("../Trial3/Arduino/GPSLOG1.csv").drop_duplicates()
newTS = pd.to_datetime(pd.to_datetime(df_arduino["Date"] + " " + df_arduino["Time"]))
new_df_arduino = pd.concat([pd.DataFrame({'TimeStamps': newTS}),df_arduino.loc[:, ['GPS','Latitude', 'Longitude']]], axis=1)

# Grouping the data as per different gps sensors and sorting as per timestamps
dfarduinogps1 = new_df_arduino[new_df_arduino["GPS"].values == "GPS1"].reset_index().sort_values(by="TimeStamps")
dfarduinogps2 = new_df_arduino[new_df_arduino["GPS"].values == "GPS2"].reset_index().sort_values(by="TimeStamps")
dfarduinogps3 = new_df_arduino[new_df_arduino["GPS"].values == "GPS3"].reset_index().sort_values(by="TimeStamps")

new_new_arduino = pd.merge(dfarduinogps1, dfarduinogps2, on='TimeStamps', how='outer')
new_new_arduino = pd.merge(new_new_arduino, dfarduinogps3, on='TimeStamps', how='outer')

new_new_arduino.rename(columns={'GPS_x': 'GPS1', 'Latitude_x':'Latitude1','Longitude_x':'Longitude1',
                               'GPS_y': 'GPS1', 'Latitude_y':'Latitude2','Longitude_y':'Longitude2',
                               'GPS': 'GPS3', 'Latitude':'Latitude3','Longitude':'Longitude3'}, inplace = True)

new_new_arduino = convert_to_utm(new_new_arduino, 'Latitude1', 'Longitude1')
new_new_arduino = convert_to_utm(new_new_arduino, 'Latitude2', 'Longitude2')
new_new_arduino = convert_to_utm(new_new_arduino, 'Latitude3', 'Longitude3')


# function to convert gps coordinates into UTM coordinates
def convert_to_utm(df, lat_col, long_col):
    utm_proj = pyproj.Proj(proj='utm', zone=32, ellps='WGS84') # set UTM projection with zone number and ellipsoid
    utm_coords = []
    for lat, lon in zip(df[lat_col], df[long_col]):
        easting, northing = utm_proj(lon, lat) # convert latitude and longitude to UTM easting and northing
        utm_coords.append((easting, northing))
    df[lat_col + '_utm_easting'], df[long_col+'_utm_northing'] = zip(*utm_coords) # add new columns to the DataFrame
    return df

# function to get angle between the subsequent coordinates
def angle_between_coordinates(lat1, lon1, lat2, lon2, lat3, lon3, lat4, lon4):
    vector1 = np.array([lat1-lat2, lon1-lon2])
    vector2 = np.array([lat3-lat4, lon3-lon4])
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    if magnitude1 == 0 or magnitude2 == 0:
        return np.nan
    
    dot_product = np.dot(vector1, vector2)
    cos_angle = dot_product / (magnitude1 * magnitude2)
    angle = np.arccos(cos_angle)
    return np.degrees(angle)